# Multiple Seldon Core Operators

This notebook illustrate how multiple Seldon Core Operators can share the same cluster. In particular:

  * A Namespaced Operator that only manages Seldon Deployments inside its namespace. Only needs Role RBAC and Namespace labeled with `seldon.io/controller-id`
  * A Clusterwide Operator that manges SeldonDeployment with a matching `seldon.io/controller-id` label.
  * A Clusterwide Operator that manages Seldon Deployments not handled by the above.

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

## Namespaced Seldon Core Operator

In [1]:
!kubectl create namespace seldon-ns1

namespace/seldon-ns1 created


In [2]:
!kubectl label namespace seldon-ns1 seldon.io/controller-id=seldon-ns1

namespace/seldon-ns1 labeled


In [2]:
VERSION=!cat ../version.txt
VERSION=VERSION[0]
VERSION

'1.19.0-dev'

In [3]:
from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

In [3]:
!helm install seldon-namespaced  ../helm-charts/seldon-core-operator  \
    --set singleNamespace=true \
    --set image.pullPolicy=IfNotPresent \
    --set usageMetrics.enabled=false \
    --set crd.create=false \
    --namespace seldon-ns1 \
    --wait

NAME: seldon-namespaced
LAST DEPLOYED: Thu Dec  4 10:08:54 2025
NAMESPACE: seldon-ns1
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [4]:
!kubectl rollout status deployment/seldon-controller-manager -n seldon-ns1

deployment "seldon-controller-manager" successfully rolled out


In [14]:
%%writetemplate resources/model.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:{VERSION}
          name: classifier
    graph:
      name: classifier
      type: MODEL
      endpoint:
        type: REST
    name: example
    replicas: 1

In [9]:
!kubectl create -f resources/model.yaml -n seldon-ns1

seldondeployment.machinelearning.seldon.io/seldon-model created


In [12]:
!kubectl wait sdep/seldon-model \
  --for=condition=ready \
  --timeout=120s \
  -n seldon-ns1

seldondeployment.machinelearning.seldon.io/seldon-model condition met


In [14]:
NAME = !kubectl get sdep -n seldon-ns1 -o jsonpath='{.items[0].metadata.name}'
assert NAME[0] == "seldon-model"

In [15]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon-ns1

Context "kind-kind" modified.


In [16]:
!kubectl delete -f resources/model.yaml -n seldon-ns1

seldondeployment.machinelearning.seldon.io "seldon-model" deleted


In [17]:
!helm delete seldon-namespaced

release "seldon-namespaced" uninstalled


## Label Focused Seldon Core Operator

 * We set `crd.create=false` as the CRD already exists in the cluster.
 * We set `controllerId=seldon-id1`. SeldonDeployments with this label will be managed.

In [18]:
!kubectl create namespace seldon-id1

namespace/seldon-id1 created


In [19]:
!helm install seldon-controllerid  ../helm-charts/seldon-core-operator  \
    --set singleNamespace=false \
    --set image.pullPolicy=IfNotPresent \
    --set usageMetrics.enabled=false \
    --set crd.create=false \
    --set controllerId=seldon-id1 \
    --namespace seldon-id1 \
    --wait

NAME: seldon-controllerid
LAST DEPLOYED: Thu Dec  4 10:13:08 2025
NAMESPACE: seldon-id1
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [20]:
!kubectl rollout status deployment/seldon-controller-manager -n seldon-id1

deployment "seldon-controller-manager" successfully rolled out


In [4]:
%%writetemplate resources/model_controller_id.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    seldon.io/controller-id: seldon-id1
  name: test-c1
spec:
  name: test-c1
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:{VERSION}
          imagePullPolicy: IfNotPresent
          name: classifier
          resources:
            requests:
              memory: 1Mi
        terminationGracePeriodSeconds: 1
    graph:
      children: []
      endpoint:
        type: REST
      name: classifier
      type: MODEL
    labels:
      version: v1
    name: example
    replicas: 1

In [5]:
!pygmentize resources/model_controller_id.yaml

apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    seldon.io/controller-id: seldon-id1
  name: test-c1
spec:
  name: test-c1
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:1.19.0-dev
          imagePullPolicy: IfNotPresent
          name: classifier
          resources:
            requests:
              memory: 1Mi
        terminationGracePeriodSeconds: 1
    graph:
      children: []
      endpoint:
        type: REST
      name: classifier
      type: MODEL
    labels:
      version: v1
    name: example
    replicas: 1


In [6]:
!kubectl create -f resources/model_controller_id.yaml -n default

seldondeployment.machinelearning.seldon.io/test-c1 created


In [10]:
!kubectl wait sdep/test-c1 \
  --for=condition=ready \
  --timeout=120s \
  -n default

seldondeployment.machinelearning.seldon.io/test-c1 condition met


In [11]:
!kubectl get sdep -n default

NAME      AGE
test-c1   26s


In [12]:
NAME = !kubectl get sdep -n default -o jsonpath='{.items[0].metadata.name}'
assert NAME[0] == "test-c1"

In [13]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon-id1

Context "kind-kind" modified.


In [14]:
!kubectl delete -f resources/model_controller_id.yaml -n default

seldondeployment.machinelearning.seldon.io "test-c1" deleted


In [15]:
!helm delete seldon-controllerid

release "seldon-controllerid" uninstalled
